# Input EDA
Exploratory Data Analysis of the Input Files

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import regex

print(pd.__version__)

1.3.5


In [3]:
# TODO: find a way to automatically pair input with output folder

inputdir_0 = 'data/input/'

sampledf_files = glob.glob(os.path.join(inputdir_0, '**/sampledataframe*.csv'), recursive=True)
solution_files = glob.glob(os.path.join(inputdir_0, '**/stocksolutions*.csv'), recursive=True)
print(sampledf_files)
print(solution_files)

# place all sample input file data into one dataframe
raw_sampledf = None
for f in sampledf_files:
    filename = f[len(inputdir_0):]
    batchname = os.path.dirname(f)[len(inputdir_0):]
    file_df = pd.read_csv(f)
    file_df.insert(0, 'file', filename, allow_duplicates=True) # record the file this data came from
    file_df.insert(1, 'batch', batchname, allow_duplicates=True) # record the batch this data came from
    raw_sampledf = pd.concat([raw_sampledf, file_df])

# place all solution input file data into one dataframe
raw_solutiondf = None
for f in solution_files:
    filename = f[len(inputdir_0):]
    batchname = os.path.dirname(f)[len(inputdir_0):]
    file_df = pd.read_csv(f)
    file_df.insert(0, 'file', filename, allow_duplicates=True) # record the file this data came from
    file_df.insert(1, 'batch', batchname, allow_duplicates=True) # record the batch this data came from
    raw_solutiondf = pd.concat([raw_solutiondf, file_df])

['data/input/20220414_Film_PL_Check/sampledataframe_WBG Molecular Sieve Dried Solvent.csv', 'data/input/20220428_Excess_PbI2/sampledataframe_PbX2 Loading Optimization.csv', 'data/input/20220502_Excess_PbI2/sampledataframe_PbX2 Loading Optimization.csv', 'data/input/20220503_Drop_optimization/sampledataframe_Drop_optimization.csv', 'data/input/20220504_PIN_Half_Cells/sampledataframe_20220504_PIN_Half_Cells.csv', 'data/input/20220505_Annealing_Optimization/sampledataframe_20220505_Annealing_Optimization.csv', 'data/input/20220506_P3AT_Spin_Optimization/sampledataframe_20220506_P3AT_Spin_Optimization.csv', 'data/input/20220506_PL_Holdout/sampledataframe_PL_Holdout_halfcells.csv', 'data/input/20220510_3xHalide_MACl_films/sampledataframe_20220510_3xHalide_MACl_films.csv', 'data/input/20220510_3xHalide_MACl_films/demo/sampledataframe_20220504_PIN_Half_Cells.csv', 'data/input/20220510_3xHalide_MACl_halfcells/sampledataframe_20220510_3xHalide_MACl_halfcells.csv', 'data/input/20220510_3xHalide_

# Analyze SampleDataframe

In [4]:
raw_sampledf

,file,batch,Unnamed: 0,name,storage_tray,storage_slot,substrate,worklist,spincoat0_drop0_time,spincoat0_drop0_height,...,spincoat4_drop1_solvent_dict,spincoat2_drop1_time,spincoat2_drop1_height,spincoat2_drop1_rate,spincoat2_drop1_volume,spincoat2_drop1_molarity,spincoat2_drop1_solutes,spincoat2_drop1_solutes_dict,spincoat2_drop1_solvent,spincoat2_drop1_solvent_dict
0,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,0,sample0,Tray1,A1,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,1,sample1,Tray1,A2,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,2,sample2,Tray1,A3,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,3,sample3,Tray1,A4,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,4,sample4,Tray1,A5,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,26,sample26,Tray2,F2,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,27,sample27,Tray2,F3,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,28,sample28,Tray2,F4,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,29,sample29,Tray2,F5,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# first, reset index and fix columns
sample_input_df = ( raw_sampledf.reset_index()
    .rename(columns={'index': 'sample_num'})
    .drop(columns='Unnamed: 0')
)
sample_input_df

,sample_num,file,batch,name,storage_tray,storage_slot,substrate,worklist,spincoat0_drop0_time,spincoat0_drop0_height,...,spincoat4_drop1_solvent_dict,spincoat2_drop1_time,spincoat2_drop1_height,spincoat2_drop1_rate,spincoat2_drop1_volume,spincoat2_drop1_molarity,spincoat2_drop1_solutes,spincoat2_drop1_solutes_dict,spincoat2_drop1_solvent,spincoat2_drop1_solvent_dict
0,0,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample0,Tray1,A1,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample1,Tray1,A2,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample2,Tray1,A3,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample3,Tray1,A4,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample4,Tray1,A5,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,26,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,sample26,Tray2,F2,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1260,27,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,sample27,Tray2,F3,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1261,28,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,sample28,Tray2,F4,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262,29,20221015_3A3X_B10/B10_2_Char/sampledataframe_B...,20221015_3A3X_B10/B10_2_Char,sample29,Tray2,F5,PSK,"[{""name"": ""characterize"", ""start"": NaN, ""id"": ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sample_input_df.columns

Index(['sample_num', 'file', 'batch', 'name', 'storage_tray', 'storage_slot',
       'substrate', 'worklist', 'spincoat0_drop0_time',
       'spincoat0_drop0_height', 'spincoat0_drop0_rate',
       'spincoat0_drop0_volume', 'spincoat0_drop0_molarity',
       'spincoat0_drop0_solutes', 'spincoat0_drop0_solutes_dict',
       'spincoat0_drop0_solvent', 'spincoat0_drop0_solvent_dict',
       'spincoat0_drop1_time', 'spincoat0_drop1_height',
       'spincoat0_drop1_rate', 'spincoat0_drop1_volume',
       'spincoat0_drop1_molarity', 'spincoat0_drop1_solutes',
       'spincoat0_drop1_solutes_dict', 'spincoat0_drop1_solvent',
       'spincoat0_drop1_solvent_dict', 'spincoat1_drop0_time',
       'spincoat1_drop0_height', 'spincoat1_drop0_rate',
       'spincoat1_drop0_volume', 'spincoat1_drop0_molarity',
       'spincoat1_drop0_solutes', 'spincoat1_drop0_solutes_dict',
       'spincoat1_drop0_solvent', 'spincoat1_drop0_solvent_dict',
       'spincoat1_drop1_time', 'spincoat1_drop1_height',
    

In [7]:
# see number of samples in each batch
pd.set_option('max_columns', None)
sample_input_df.groupby('batch').apply(lambda group: group.count())

,sample_num,file,batch,name,storage_tray,storage_slot,substrate,worklist,spincoat0_drop0_time,spincoat0_drop0_height,spincoat0_drop0_rate,spincoat0_drop0_volume,spincoat0_drop0_molarity,spincoat0_drop0_solutes,spincoat0_drop0_solutes_dict,spincoat0_drop0_solvent,spincoat0_drop0_solvent_dict,spincoat0_drop1_time,spincoat0_drop1_height,spincoat0_drop1_rate,spincoat0_drop1_volume,spincoat0_drop1_molarity,spincoat0_drop1_solutes,spincoat0_drop1_solutes_dict,spincoat0_drop1_solvent,spincoat0_drop1_solvent_dict,spincoat1_drop0_time,spincoat1_drop0_height,spincoat1_drop0_rate,spincoat1_drop0_volume,spincoat1_drop0_molarity,spincoat1_drop0_solutes,spincoat1_drop0_solutes_dict,spincoat1_drop0_solvent,spincoat1_drop0_solvent_dict,spincoat1_drop1_time,spincoat1_drop1_height,spincoat1_drop1_rate,spincoat1_drop1_volume,spincoat1_drop1_molarity,spincoat1_drop1_solutes,spincoat1_drop1_solutes_dict,spincoat1_drop1_solvent,spincoat1_drop1_solvent_dict,spincoat2_drop0_time,spincoat2_drop0_height,spincoat2_drop0_rate,spincoat2_drop0_volume,spincoat2_drop0_molarity,spincoat2_drop0_solutes,spincoat2_drop0_solutes_dict,spincoat2_drop0_solvent,spincoat2_drop0_solvent_dict,spincoat3_drop0_time,spincoat3_drop0_height,spincoat3_drop0_rate,spincoat3_drop0_volume,spincoat3_drop0_molarity,spincoat3_drop0_solutes,spincoat3_drop0_solutes_dict,spincoat3_drop0_solvent,spincoat3_drop0_solvent_dict,spincoat4_drop0_time,spincoat4_drop0_height,spincoat4_drop0_rate,spincoat4_drop0_volume,spincoat4_drop0_molarity,spincoat4_drop0_solutes,spincoat4_drop0_solutes_dict,spincoat4_drop0_solvent,spincoat4_drop0_solvent_dict,spincoat4_drop1_time,spincoat4_drop1_height,spincoat4_drop1_rate,spincoat4_drop1_volume,spincoat4_drop1_molarity,spincoat4_drop1_solutes,spincoat4_drop1_solutes_dict,spincoat4_drop1_solvent,spincoat4_drop1_solvent_dict,spincoat2_drop1_time,spincoat2_drop1_height,spincoat2_drop1_rate,spincoat2_drop1_volume,spincoat2_drop1_molarity,spincoat2_drop1_solutes,spincoat2_drop1_solutes_dict,spincoat2_drop1_solvent,spincoat2_drop1_solvent_dict
batch,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20220414_Film_PL_Check,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220428_Excess_PbI2,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,0,42,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220502_Excess_PbI2,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,0,18,18,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220503_Drop_optimization,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,0,26,26,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220504_PIN_Half_Cells,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,0,0,0,0,0,0,0,0,0,30,30,30,30,30,30,30,30,30,30,30,30,30,30,0,30,30,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220505_Annealing_Optimization,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,0,18,18,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220506_P3AT_Spin_Optimization,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220506_PL_Holdout,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20220510_3xHalide_MACl_films,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,

In [8]:
# now see number of unique parameters over all batches
pd.set_option('max_rows', 100)
sample_input_df.apply(lambda group: group.unique()).to_dict()

{'sample_num': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59]),
 'file': array(['20220414_Film_PL_Check/sampledataframe_WBG Molecular Sieve Dried Solvent.csv',
        '20220428_Excess_PbI2/sampledataframe_PbX2 Loading Optimization.csv',
        '20220502_Excess_PbI2/sampledataframe_PbX2 Loading Optimization.csv',
        '20220503_Drop_optimization/sampledataframe_Drop_optimization.csv',
        '20220504_PIN_Half_Cells/sampledataframe_20220504_PIN_Half_Cells.csv',
        '20220505_Annealing_Optimization/sampledataframe_20220505_Annealing_Optimization.csv',
        '20220506_P3AT_Spin_Optimization/sampledataframe_20220506_P3AT_Spin_Optimization.csv',
        '20220506_PL_Holdout/sampledataframe_PL_Holdout_halfcells.csv',
        '20220510_3xHalide_MACl_films/sampl

In [9]:
# get each batch's unique spincoating parameters
x = sample_input_df.filter(like='spincoat')
x.loc[:, 'batch'] = sample_input_df['batch']
pd.DataFrame(x.groupby('batch').apply(lambda group: group.apply(
    lambda col: col.unique()
))[0])

/home/kcw/anaconda3/envs/fenning/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


0
batch                                                        
20220414_Film_PL_Check       spincoat0_drop0_time      [-1.0]
                             spincoat0_drop0_height     [2.0]
                             spincoat0_drop0_rate      [80.0]
                             spincoat0_drop0_volume    [30.0]
                             spincoat0_drop0_molarity   [1.2]
...                                                       ...
20221011_3A3X_B9/B9_1_PSK    0                            NaN
20221011_3A3X_B9/B9_2_Char   0                            NaN
20221015_3A3X_B10/B10_0_HTL  0                            NaN
20221015_3A3X_B10/B10_1_PSK  0                            NaN
20221015_3A3X_B10/B10_2_Char 0                            NaN

[1749 rows x 1 columns]

Some batches have multiple spincoats.

We create a column to count how many spincoats a sample undergoes.

In [39]:
# determine number of spincoats for each batch/task
# if a row does not have spincoat num i, it will have an na in the column "spincoat{i}_drop0_solvent_dict"
spin_cols = [c for c in sample_input_df.columns if 'spincoat' in c and '_drop0_solvent_dict' in c]
print(spin_cols)
sample_input_df['spincoat_count'] = sample_input_df.apply(
    lambda row : row[spin_cols].notna().sum(), axis=1)
pd.reset_option('max_rows')
sample_input_df

['spincoat0_drop0_solvent_dict', 'spincoat1_drop0_solvent_dict', 'spincoat2_drop0_solvent_dict', 'spincoat3_drop0_solvent_dict', 'spincoat4_drop0_solvent_dict']


,sample_num,file,batch,name,storage_tray,storage_slot,substrate,worklist,spincoat0_drop0_time,spincoat0_drop0_height,spincoat0_drop0_rate,spincoat0_drop0_volume,spincoat0_drop0_molarity,spincoat0_drop0_solutes,spincoat0_drop0_solutes_dict,spincoat0_drop0_solvent,spincoat0_drop0_solvent_dict,spincoat0_drop1_time,spincoat0_drop1_height,spincoat0_drop1_rate,spincoat0_drop1_volume,spincoat0_drop1_molarity,spincoat0_drop1_solutes,spincoat0_drop1_solutes_dict,spincoat0_drop1_solvent,spincoat0_drop1_solvent_dict,spincoat1_drop0_time,spincoat1_drop0_height,spincoat1_drop0_rate,spincoat1_drop0_volume,spincoat1_drop0_molarity,spincoat1_drop0_solutes,spincoat1_drop0_solutes_dict,spincoat1_drop0_solvent,spincoat1_drop0_solvent_dict,spincoat1_drop1_time,spincoat1_drop1_height,spincoat1_drop1_rate,spincoat1_drop1_volume,spincoat1_drop1_molarity,spincoat1_drop1_solutes,spincoat1_drop1_solutes_dict,spincoat1_drop1_solvent,spincoat1_drop1_solvent_dict,spincoat2_drop0_time,spincoat2_drop0_height,spincoat2_drop0_rate,spincoat2_drop0_volume,spincoat2_drop0_molarity,spincoat2_drop0_solutes,spincoat2_drop0_solutes_dict,spincoat2_drop0_solvent,spincoat2_drop0_solvent_dict,spincoat3_drop0_time,spincoat3_drop0_height,spincoat3_drop0_rate,spincoat3_drop0_volume,spincoat3_drop0_molarity,spincoat3_drop0_solutes,spincoat3_drop0_solutes_dict,spincoat3_drop0_solvent,spincoat3_drop0_solvent_dict,spincoat4_drop0_time,spincoat4_drop0_height,spincoat4_drop0_rate,spincoat4_drop0_volume,spincoat4_drop0_molarity,spincoat4_drop0_solutes,spincoat4_drop0_solutes_dict,spincoat4_drop0_solvent,spincoat4_drop0_solvent_dict,spincoat4_drop1_time,spincoat4_drop1_height,spincoat4_drop1_rate,spincoat4_drop1_volume,spincoat4_drop1_molarity,spincoat4_drop1_solutes,spincoat4_drop1_solutes_dict,spincoat4_drop1_solvent,spincoat4_drop1_solvent_dict,spincoat2_drop1_time,spincoat2_drop1_height,spincoat2_drop1_rate,spincoat2_drop1_volume,spincoat2_drop1_molarity,spincoat2_drop1_solutes,spincoat2_drop1_solutes_dict,spincoat2_drop1_solvent,spincoat2_drop1_solvent_dict,spincoat_count
0,0,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample0,Tray1,A1,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,80.0,30.0,1.2,FA0.78_Cs0.17_MA0.05_Pb1.09_I2.62_Br0.491_Cl0.164,"{""FA"": 0.9359999999999999, ""Cs"": 0.20400000000...",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",33.0,2.0,80.0,100.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample1,Tray1,A2,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,80.0,30.0,1.2,FA0.78_Cs0.17_MA0.05_Pb1.09_I2.62_Br0.491_Cl0.164,"{""FA"": 0.9359999999999999, ""Cs"": 0.20400000000...",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",33.0,2.0,80.0,100.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,20220414_Film_PL_Check/sampledataframe_WBG Mol...,20220414_Film_PL_Check,sample2,Tray1,A3,glass,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-1.0,2.0,80.0,30.0,1.2,FA0.78_Cs0.17_MA0.05_Pb1.09_I2.62_Br0.491_Cl0.164,"{""FA"": 0.9359999999999999, ""Cs"": 0.20400000000...",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",33.0,2.0,80.0,100.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [31]:
sample_input_df.groupby('batch').apply(lambda group: group['spincoat_count'].unique())

batch
20220414_Film_PL_Check                                               [1]
20220428_Excess_PbI2                                                 [1]
20220502_Excess_PbI2                                                 [1]
20220503_Drop_optimization                                           [1]
20220504_PIN_Half_Cells                                              [2]
20220505_Annealing_Optimization                                      [1]
20220506_P3AT_Spin_Optimization                                      [1]
20220506_PL_Holdout                                                  [0]
20220510_3xHalide_MACl_films                                         [1]
20220510_3xHalide_MACl_films/demo                                    [1]
20220510_3xHalide_MACl_halfcells                                     [2]
20220510_3xHalide_MACl_halfcells_charonly                            [0]
20220511_3xHalide_MACl_mix_films                                     [1]
20220516_3A3X_halfcells                      

In [41]:
# group by number of spincoats and first solute/solvent used
pd.set_option('max_rows', None)
pd.DataFrame(sample_input_df.groupby(
    ['spincoat_count', # outer group by number of spincoats
    'spincoat0_drop1_solvent_dict', # group by antisolvent (not many different ones)
    'spincoat0_drop0_solvent_dict', 'spincoat0_drop0_solutes_dict', # then group by first solution
    'batch'], #then give batch name
    dropna=False
    ).size().rename('num_samples'))

num_samples
spincoat_count spincoat0_drop1_solvent_dict            spincoat0_drop0_solvent_dict        spincoat0_drop0_solutes_dict                       batch                                                          
0              NaN                                     NaN                                 NaN                                                20220506_PL_Holdout                                           3
                                                                                                                                              20220510_3xHalide_MACl_halfcells_charonly                    16
                                                                                                                                              20220525_XEOL_Charonly                                       12
                                                                                                                                              20220727_3A3X_Xu_halfcells                                   45
                                                                                                                                              20220803_FAPbI_Char_only                                      1
                                                                                                                                              20220807_3A3X_T2_char_only                                   44
                                                                                                                                              20220807_Char_test                                            1
                                                                                                                                              20220829_3A3X_B5_postHTL                                     10
                                                                                                                                              20220901_Charonly_Jack                                        4
                                                                                                                                              20220927_3A3X_B8/B8_2_Char                                   32
                                                                                                                                              20221011_3A3X_B9/B9_2_Char                                   31
                                                                                                                                              20221015_3A3X_B10/B10_2_Char                                 31
1              {"Chlorobenzene": 1.0}                  {"DMF": 0.8, "DMSO": 0.2}           {"Wolf-Recipe-PSK": 1.7}                           20220726_3A2X_Halfcells                                      10
               {"MethylAcetate": 0.5, "StockTip": 0.5} {"DMF": 0.75, "DMSO": 0.25}         {"MAI)": 0.35, "(MACl": 0.7}                       20220517_3A3X_AS_tip_optimization                             8
               {"MethylAcetate": 0.5, "WideTip": 0.5}  {"DMF": 0.75, "DMSO": 0.25}         {"MAI)": 0.35, "(MACl": 0.7}                       20220517_3A3X_AS_tip_optimization                             8
                                                                                           {"Xu-Recipe-PSK": 1.4}                             20220523_3A3X_Xu-recipe                                      32
                                                                                                                                              20220523_3A3X_Xu-recipe-prerun                                6
               {"MethylAcetate": 1.0}                  {"DMF": 0.6, "DMSO": 0.2, "b": 0.2} {"FA": 0.9359999999999999, "Cs": 0.264, "Pb": 1... 20220414_Film_PL_Check                                        3
                                                       {"DMF": 0.75, "DMSO

In [88]:
# for batches with more than one spincoat, show them individually
# 2 spincoat batches 

spins_2 = pd.DataFrame(sample_input_df[sample_input_df['spincoat_count'] == 2]
    .groupby(['spincoat1_drop1_solvent_dict', # first group by antisolvent
        'spincoat1_drop0_solvent_dict', 'spincoat1_drop0_solutes_dict', # then by actual film solution
        'spincoat0_drop0_solvent_dict', 'spincoat0_drop0_solutes_dict',
        'batch'],
        dropna=False)
    .size().rename('num_samples')
)
spins_2

num_samples
spincoat1_drop1_solvent_dict spincoat1_drop0_solvent_dict spincoat1_drop0_solutes_dict                       spincoat0_drop0_solvent_dict spincoat0_drop0_solutes_dict    batch                                        
{"Chlorobenzene": 1.0}       {"DMF": 0.8, "DMSO": 0.2}    {"Wolf-Recipe-PSK": 1.7}                           {"IPA": 1.0}                 {"SAMS": 0.0006, "HTL": 0.0006} 20220726_3A2X_Halfcells                    10
{"MethylAcetate": 1.0}       {"DMF": 0.75, "DMSO": 0.25}  {"FA.": 109.19999999999999, "MA.": 7.0, "Cs.": ... {"IPA": 1.0}                 {"SAMS": 0.0006, "HTL": 0.0006} 20220504_PIN_Half_Cells                    30
                                                          {"MACl": 1.4, "": 1.4}                             {"IPA": 1.0}                 {"SAMS": 0.0006, "HTL": 0.0006} 20220510_3xHalide_MACl_halfcells            8
                                                          {"MACl": 1.4}                                      {"IPA": 1.0}                 {"SAMS": 0.0006, "HTL": 0.0006} 20220510_3xHalide_MACl_halfcells            8
                                                          {"Xu-Recipe-PSK": 1.4}                             {"Ethanol": 1.0}             {"SAMS": 0.001, "HTL": 0.001}   20220807_3A3X_T2                           45
                                                                                                                                                                          20220815_3A3X_A3                           60
                                                                                                                                                                          20220825_3A3X_B4                           40
                                                                                                                                          {"SAMS": 0.002, "HTL": 0.002}   20220829_3A3X_B5                           45
                                                                                                                                                                          20220831_3A3X_B6                            4

In [114]:
# 3 spincoat batches 

spins_3 = sample_input_df[sample_input_df['spincoat_count'] == 3]
count = spins_3.groupby('batch').size().rename('num_samples')
spins_3 = spins_3.join(count, on='batch')
spins_3 = spins_3.drop_duplicates(spins_3.filter(like='spincoat').columns.to_list())
spins_3

,sample_num,file,batch,name,storage_tray,storage_slot,substrate,worklist,spincoat0_drop0_time,spincoat0_drop0_height,spincoat0_drop0_rate,spincoat0_drop0_volume,spincoat0_drop0_molarity,spincoat0_drop0_solutes,spincoat0_drop0_solutes_dict,spincoat0_drop0_solvent,spincoat0_drop0_solvent_dict,spincoat0_drop1_time,spincoat0_drop1_height,spincoat0_drop1_rate,spincoat0_drop1_volume,spincoat0_drop1_molarity,spincoat0_drop1_solutes,spincoat0_drop1_solutes_dict,spincoat0_drop1_solvent,spincoat0_drop1_solvent_dict,spincoat1_drop0_time,spincoat1_drop0_height,spincoat1_drop0_rate,spincoat1_drop0_volume,spincoat1_drop0_molarity,spincoat1_drop0_solutes,spincoat1_drop0_solutes_dict,spincoat1_drop0_solvent,spincoat1_drop0_solvent_dict,spincoat1_drop1_time,spincoat1_drop1_height,spincoat1_drop1_rate,spincoat1_drop1_volume,spincoat1_drop1_molarity,spincoat1_drop1_solutes,spincoat1_drop1_solutes_dict,spincoat1_drop1_solvent,spincoat1_drop1_solvent_dict,spincoat2_drop0_time,spincoat2_drop0_height,spincoat2_drop0_rate,spincoat2_drop0_volume,spincoat2_drop0_molarity,spincoat2_drop0_solutes,spincoat2_drop0_solutes_dict,spincoat2_drop0_solvent,spincoat2_drop0_solvent_dict,spincoat3_drop0_time,spincoat3_drop0_height,spincoat3_drop0_rate,spincoat3_drop0_volume,spincoat3_drop0_molarity,spincoat3_drop0_solutes,spincoat3_drop0_solutes_dict,spincoat3_drop0_solvent,spincoat3_drop0_solvent_dict,spincoat4_drop0_time,spincoat4_drop0_height,spincoat4_drop0_rate,spincoat4_drop0_volume,spincoat4_drop0_molarity,spincoat4_drop0_solutes,spincoat4_drop0_solutes_dict,spincoat4_drop0_solvent,spincoat4_drop0_solvent_dict,spincoat4_drop1_time,spincoat4_drop1_height,spincoat4_drop1_rate,spincoat4_drop1_volume,spincoat4_drop1_molarity,spincoat4_drop1_solutes,spincoat4_drop1_solutes_dict,spincoat4_drop1_solvent,spincoat4_drop1_solvent_dict,spincoat2_drop1_time,spincoat2_drop1_height,spincoat2_drop1_rate,spincoat2_drop1_volume,spincoat2_drop1_molarity,spincoat2_drop1_solutes,spincoat2_drop1_solutes_dict,spincoat2_drop1_solvent,spincoat2_drop1_solvent_dict,spincoat_count,num_samples
255,16,20220516_3A3X_halfcells/sampledataframe_202205...,20220516_3A3X_halfcells,sample16,Tray2,D2,FTO,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,2.0,80.0,30.0,1.4,MAI)0.25_(MACl0.5,"{""MAI)"": 0.35, ""(MACl"": 0.7}",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,2.0,100.0,75.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",3,16
341,16,20220524_3A3X_Xu-halfcells/sampledataframe_202...,20220524_3A3X_Xu-halfcells,sample16,Tray2,D2,FTO,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,2.0,80.0,30.0,1.4,Xu-Recipe-PSK,"{""Xu-Recipe-PSK"": 1.4}",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,0.5,450.0,75.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",3,32


In [115]:
# 5 spincoat batches 

spins_5 = sample_input_df[sample_input_df['spincoat_count'] == 5]
count = spins_5.groupby('batch').size().rename('num_samples')
spins_5 = spins_5.join(count, on='batch')
spins_5 = spins_5.drop_duplicates(spins_5.filter(like='spincoat').columns.to_list())
spins_5

,sample_num,file,batch,name,storage_tray,storage_slot,substrate,worklist,spincoat0_drop0_time,spincoat0_drop0_height,spincoat0_drop0_rate,spincoat0_drop0_volume,spincoat0_drop0_molarity,spincoat0_drop0_solutes,spincoat0_drop0_solutes_dict,spincoat0_drop0_solvent,spincoat0_drop0_solvent_dict,spincoat0_drop1_time,spincoat0_drop1_height,spincoat0_drop1_rate,spincoat0_drop1_volume,spincoat0_drop1_molarity,spincoat0_drop1_solutes,spincoat0_drop1_solutes_dict,spincoat0_drop1_solvent,spincoat0_drop1_solvent_dict,spincoat1_drop0_time,spincoat1_drop0_height,spincoat1_drop0_rate,spincoat1_drop0_volume,spincoat1_drop0_molarity,spincoat1_drop0_solutes,spincoat1_drop0_solutes_dict,spincoat1_drop0_solvent,spincoat1_drop0_solvent_dict,spincoat1_drop1_time,spincoat1_drop1_height,spincoat1_drop1_rate,spincoat1_drop1_volume,spincoat1_drop1_molarity,spincoat1_drop1_solutes,spincoat1_drop1_solutes_dict,spincoat1_drop1_solvent,spincoat1_drop1_solvent_dict,spincoat2_drop0_time,spincoat2_drop0_height,spincoat2_drop0_rate,spincoat2_drop0_volume,spincoat2_drop0_molarity,spincoat2_drop0_solutes,spincoat2_drop0_solutes_dict,spincoat2_drop0_solvent,spincoat2_drop0_solvent_dict,spincoat3_drop0_time,spincoat3_drop0_height,spincoat3_drop0_rate,spincoat3_drop0_volume,spincoat3_drop0_molarity,spincoat3_drop0_solutes,spincoat3_drop0_solutes_dict,spincoat3_drop0_solvent,spincoat3_drop0_solvent_dict,spincoat4_drop0_time,spincoat4_drop0_height,spincoat4_drop0_rate,spincoat4_drop0_volume,spincoat4_drop0_molarity,spincoat4_drop0_solutes,spincoat4_drop0_solutes_dict,spincoat4_drop0_solvent,spincoat4_drop0_solvent_dict,spincoat4_drop1_time,spincoat4_drop1_height,spincoat4_drop1_rate,spincoat4_drop1_volume,spincoat4_drop1_molarity,spincoat4_drop1_solutes,spincoat4_drop1_solutes_dict,spincoat4_drop1_solvent,spincoat4_drop1_solvent_dict,spincoat2_drop1_time,spincoat2_drop1_height,spincoat2_drop1_rate,spincoat2_drop1_volume,spincoat2_drop1_molarity,spincoat2_drop1_solutes,spincoat2_drop1_solutes_dict,spincoat2_drop1_solvent,spincoat2_drop1_solvent_dict,spincoat_count,num_samples
239,0,20220516_3A3X_halfcells/sampledataframe_202205...,20220516_3A3X_halfcells,sample0,Tray2,A1,FTO,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.0,80.0,30.0,1.0,IPA,"{""IPA"": 1.0}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",10.0,2.0,80.0,30.0,1.0,IPA,"{""IPA"": 1.0}",IPA,"{""IPA"": 1.0}",-2.0,2.0,80.0,30.0,1.4,MAI)0.25_(MACl0.5,"{""MAI)"": 0.35, ""(MACl"": 0.7}",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",45.0,2.0,100.0,75.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,16
325,0,20220524_3A3X_Xu-halfcells/sampledataframe_202...,20220524_3A3X_Xu-halfcells,sample0,Tray2,A1,FTO,"[{""name"": ""spincoat"", ""start"": NaN, ""id"": ""spi...",-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.0,80.0,30.0,1.0,IPA,"{""IPA"": 1.0}",IPA,"{""IPA"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,2.0,80.0,30.0,0.0006,SAMS_HTL,"{""SAMS"": 0.0006, ""HTL"": 0.0006}",IPA,"{""IPA"": 1.0}",10.0,2.0,80.0,30.0,1.0,IPA,"{""IPA"": 1.0}",IPA,"{""IPA"": 1.0}",-2.0,2.0,80.0,30.0,1.4,Xu-Recipe-PSK,"{""Xu-Recipe-PSK"": 1.4}",DMF0.75_DMSO0.25,"{""DMF"": 0.75, ""DMSO"": 0.25}",30.0,0.5,450.0,75.0,0.0,NaN,{},MethylAcetate,"{""MethylAcetate"": 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,32


# Analyze StockSolutions

In [125]:
raw_solutiondf.head()

,file,batch,Labware,Well,Volume (uL),Solutes,Molarity (M),Solvent
0,20220414_Film_PL_Check/stocksolutions_WBG Mole...,20220414_Film_PL_Check,4mL_Tray1,A1,140,"{'FA': 0.9359999999999999, 'Cs': 0.20400000000...",1.2,"{'DMF': 0.75, 'DMSO': 0.25}"
1,20220414_Film_PL_Check/stocksolutions_WBG Mole...,20220414_Film_PL_Check,4mL_Tray1,D1,650,{},0.0,{'MethylAcetate': 1.0}
2,20220414_Film_PL_Check/stocksolutions_WBG Mole...,20220414_Film_PL_Check,4mL_Tray1,A2,140,"{'FA': 0.9359999999999999, 'Cs': 0.264, 'Pb': ...",1.2,"{'DMF': 0.6, 'DMSO': 0.2, 'b': 0.2}"
0,20220428_Excess_PbI2/stocksolutions_PbX2 Loadi...,20220428_Excess_PbI2,96wellplate,A1,Empty Vial,"{'FA': 0.9359999999999999, 'Cs': 0.20400000000...",1.2,"{'DMF': 0.75, 'DMSO': 0.25}"
1,20220428_Excess_PbI2/stocksolutions_PbX2 Loadi...,20220428_Excess_PbI2,96wellplate,A2,Empty Vial,"{'FA': 0.9359999999999999, 'Cs': 0.20400000000...",1.2,"{'DMF': 0.75, 'DMSO': 0.25}"


In [127]:
solution_input_df = raw_solutiondf.copy()

In [130]:
# see number of unique parameters over all batches
solution_input_df.apply(lambda col: col.unique()).to_dict()

{'file': array(['20220414_Film_PL_Check/stocksolutions_WBG Molecular Sieve Dried Solvent.csv',
        '20220428_Excess_PbI2/stocksolutions_PbX2 Loading Optimization.csv',
        '20220502_Excess_PbI2/stocksolutions_PbX2 Loading Optimization.csv',
        '20220503_Drop_optimization/stocksolutions_Drop_optimization.csv',
        '20220504_PIN_Half_Cells/stocksolutions_20220504_PIN_Half_Cells.csv',
        '20220505_Annealing_Optimization/stocksolutions_20220505_Annealing_Optimization.csv',
        '20220506_P3AT_Spin_Optimization/stocksolutions_20220506_P3AT_Spin_Optimization.csv',
        '20220510_3xHalide_MACl_films/stocksolutions_20220510_3xHalide_MACl_films.csv',
        '20220510_3xHalide_MACl_films/demo/stocksolutions_20220504_PIN_Half_Cells.csv',
        '20220510_3xHalide_MACl_halfcells/stocksolutions_20220510_3xHalide_MACl_halfcells.csv',
        '20220511_3xHalide_MACl_mix_films/stocksolutions_20220511_3xHalide_MACl_mix_films.csv',
        '20220516_3A3X_halfcells/stocksolu

In [165]:
# see the solutions used for each batch
solution_input_df.set_index(['Solvent', 'Solutes', 'batch']).sort_index().drop(columns='file')

Labware  \
Solvent                                 Solutes                                            batch                                                             
{'CB': 1.0}                             {'n': 10.0}                                        20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 4.0, 'a': 1.0}                               20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 4.0, 'b': 1.0}                               20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 4.0, 'c': 1.0}                               20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 5.0}                                         20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 6.0}                                         20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 7.0}                                         20220506_P3AT_Spin_Optimization                       4mL_Tray1   
                                        {'n': 8.0}                                         20220506_P3AT_Spin_Optimization                       4mL_Tray1   
{'Chlorobenzene': 1.0}                  {}                                                 20220726_3A2X_Halfcells                              15mL_Tray1   
{'DMF': 0.6, 'DMSO': 0.2, 'b': 0.2}     {'FA': 0.9359999999999999, 'Cs': 0.264, 'Pb': 1... 20220414_Film_PL_Check                                4mL_Tray1   
{'DMF': 0.75, 'DMSO': 0.25}             {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220428_Excess_PbI2                                 15mL_Tray1   
                                        {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220428_Excess_PbI2                                96wellplate   
                                        {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220428_Excess_PbI2                                96wellplate   
                                        {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220428_Excess_PbI2                                96wellplate   
                                        {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220428_Excess_PbI2                                96wellplate   
                                        {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220428_Excess_PbI2                                96wellplate   
                                        {'FA': 0.9359999999999999, 'Cs': 0.204000000000... 20220414_Film_PL_Check                                4mL_Tray1   
                                                                                           20220428_Excess_PbI2                                 15mL_Tray1   
                                        {'FA': 0.975, 'Cs': 0.15, 'MA': 0.125, 'Pb': 1.... 20220930 Repeat Total - Last run + control/Expe...    4mL_Tray1   
                                                                                           20220930 Repeat Total - Last run + control/Expe...    4mL_Tray1   
                                        {'FA': 0.975, 'Cs': 0.15, 'MA': 0.125, 'Pb': 1.... 20220930 Repeat Total - Last run + control/Expe...    4mL_Tray1   
                                                                                           20220930 Repeat Total - Last run + control/Expe...    4mL_Tray1   
                                        {'FA': 0.975, 'Cs': 0.15, 'MA': 0.125, 'Pb': 1.... 20220930 Repeat Total - Last run + control/Expe...    4mL_Tray1   
                                                                                           20220930 Repeat Total - Last run + control/Expe...    4mL_Tray1   
                                       

# Analyze Maestro Netlist

In [167]:
import json

maestro_files = glob.glob(os.path.join(inputdir_0, '**/maestronetlist*.json'), recursive=True)
print(maestro_files)

# place all maestro input file data into one dataframe
raw_maestrodf = None
for f in maestro_files:
    filename = f[len(inputdir_0):]
    batchname = os.path.dirname(f)[len(inputdir_0):]
    file_df = pd.read_json(f)
    file_df.insert(0, 'file', filename, allow_duplicates=True) # record the file this data came from
    file_df.insert(1, 'batch', batchname, allow_duplicates=True) # record the batch this data came from
    raw_maestrodf = pd.concat([raw_maestrodf, file_df])


['data/input/20220414_Film_PL_Check/maestronetlist_WBG Molecular Sieve Dried Solvent.json', 'data/input/20220428_Excess_PbI2/maestronetlist_PbX2 Loading Optimization.json', 'data/input/20220502_Excess_PbI2/maestronetlist_PbX2 Loading Optimization.json', 'data/input/20220503_Drop_optimization/maestronetlist_PbX2 Loading Optimization.json', 'data/input/20220504_PIN_Half_Cells/maestronetlist_20220504_PIN_Half_Cells.json', 'data/input/20220505_Annealing_Optimization/maestronetlist_20220505_Annealing_Optimization.json', 'data/input/20220506_P3AT_Spin_Optimization/maestronetlist_20220506_P3AT_Spin_Optimization.json', 'data/input/20220506_PL_Holdout/maestronetlist_PL_Holdout_halfcells.json', 'data/input/20220510_3xHalide_MACl_films/maestronetlist_20220510_3xHalide_MACl_films.json', 'data/input/20220510_3xHalide_MACl_films/demo/maestronetlist_20220504_PIN_Half_Cells.json', 'data/input/20220510_3xHalide_MACl_halfcells/maestronetlist_20220510_3xHalide_MACl_halfcells.json', 'data/input/20220510_3

In [171]:
pd.reset_option('max_rows')
raw_maestrodf.head(10)

,file,batch,baselines_required,description,hotplate_setpoints,name,samples
Brightfield,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,[0.05],Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
Darkfield,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,[0.05],Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
PL_635nm,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,"[0.1, 20, 5]",Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
Transmission,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,"[0.2, 1, 0.05, 5, 15, 0.02]",Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
Hotplate1,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,100.0,WBG Molecular Sieve Dried Solvent,NaN
sample0,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample0', 'storage_slot': {'slot': '..."
sample1,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample1', 'storage_slot': {'slot': '..."
sample2,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample2', 'storage_slot': {'slot': '..."
sample3,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample3', 'storage_slot': {'slot': '..."
sample4,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample4', 'storage_slot': {'slot': '..."


In [182]:
maestro_input_df = raw_maestrodf.reset_index().rename(columns={'index': 'task'})
maestro_input_df.head(10)

,task,file,batch,baselines_required,description,hotplate_setpoints,name,samples
0,Brightfield,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,[0.05],Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
1,Darkfield,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,[0.05],Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
2,PL_635nm,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,"[0.1, 20, 5]",Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
3,Transmission,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,"[0.2, 1, 0.05, 5, 15, 0.02]",Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,NaN
4,Hotplate1,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,100.0,WBG Molecular Sieve Dried Solvent,NaN
5,sample0,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample0', 'storage_slot': {'slot': '..."
6,sample1,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample1', 'storage_slot': {'slot': '..."
7,sample2,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample2', 'storage_slot': {'slot': '..."
8,sample3,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample3', 'storage_slot': {'slot': '..."
9,sample4,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,NaN,WBG Molecular Sieve Dried Solvent,"{'name': 'sample4', 'storage_slot': {'slot': '..."


In [197]:
maestro_input_df.columns

Index(['task', 'file', 'batch', 'baselines_required', 'description',
       'hotplate_setpoints', 'name', 'samples'],
      dtype='object')

In [205]:
# see unique parameters
#need to cast to prevent unhashable type errors for pd.unique
maestro_input_df.apply(lambda col: col.astype(str).unique()).to_dict()

{'task': array(['Brightfield', 'Darkfield', 'PL_635nm', 'Transmission',
        'Hotplate1', 'sample0', 'sample1', 'sample2', 'sample3', 'sample4',
        'sample5', 'sample10', 'sample11', 'sample12', 'sample13',
        'sample14', 'sample15', 'sample16', 'sample17', 'sample18',
        'sample19', 'sample20', 'sample21', 'sample22', 'sample23',
        'sample24', 'sample25', 'sample26', 'sample27', 'sample28',
        'sample29', 'sample30', 'sample31', 'sample32', 'sample33',
        'sample34', 'sample35', 'sample36', 'sample37', 'sample38',
        'sample39', 'sample40', 'sample41', 'sample6', 'sample7',
        'sample8', 'sample9', 'PLImaging', 'Hotplate2', 'Hotplate3',
        'sample42', 'sample43', 'sample44', 'Photostability_405nm',
        'sample45', 'sample46', 'sample47', 'sample48', 'sample49',
        'sample50', 'sample51', 'sample52', 'sample53', 'sample54',
        'sample55', 'sample56', 'sample57', 'sample58', 'sample59'],
       dtype=object),
 'file': array(

In [206]:
# see hotplates
maestro_input_df[maestro_input_df['hotplate_setpoints'].notna()]

,task,file,batch,baselines_required,description,hotplate_setpoints,name,samples
4,Hotplate1,20220414_Film_PL_Check/maestronetlist_WBG Mole...,20220414_Film_PL_Check,NaN,Moses prepped same nominal solution with bead ...,100.0,WBG Molecular Sieve Dried Solvent,NaN
15,Hotplate1,20220428_Excess_PbI2/maestronetlist_PbX2 Loadi...,20220428_Excess_PbI2,NaN,Purpose of this experiment is to optimize the ...,100.0,PbX2 Loading Optimization,NaN
62,Hotplate1,20220502_Excess_PbI2/maestronetlist_PbX2 Loadi...,20220502_Excess_PbI2,NaN,Drop conditions,100.0,PbX2 Loading Optimization,NaN
85,Hotplate1,20220503_Drop_optimization/maestronetlist_PbX2...,20220503_Drop_optimization,NaN,Drop conditions,100.0,PbX2 Loading Optimization,NaN
116,Hotplate1,20220504_PIN_Half_Cells/maestronetlist_2022050...,20220504_PIN_Half_Cells,NaN,Half-cells for Apoorva,100.0,20220504_PIN_Half_Cells,NaN
152,Hotplate1,20220505_Annealing_Optimization/maestronetlist...,20220505_Annealing_Optimization,NaN,Annealing optimization on thin films,100.0,20220505_Annealing_Optimization,NaN
153,Hotplate2,20220505_Annealing_Optimization/maestronetlist...,20220505_Annealing_Optimization,NaN,Annealing optimization on thin films,115.0,20220505_Annealing_Optimization,NaN
154,Hotplate3,20220505_Annealing_Optimization/maestronetlist...,20220505_Annealing_Optimization,NaN,Annealing optimization on thin films,130.0,20220505_Annealing_Optimization,NaN
222,Hotplate1,20220510_3xHalide_MACl_films/maestronetlist_20...,20220510_3xHalide_MACl_films,NaN,Substituting MAI with MACl,100.0,20220510_3xHalide_MACl_films,NaN
223,Hotplate2,20220510_3xHalide_MACl_films/maestronetlist_20...,20220510_3xHalide_MACl_films,NaN,Substituting MAI with MACl,115.0,20220510_3xHalide_MACl_films,NaN
